In [9]:
import tensorflow as tf
import numpy as np
import random
from sklearn.model_selection import KFold
from load_data import load_toy
from keras.layers import Dense
from copy import deepcopy
from tensorflow.keras.models import Sequential

In [32]:

dataset_name = "toy"
data_dir = "../Data/" + dataset_name


total_graph = load_toy(data_dir)


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/opt/anaconda3/envs/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3572, in run_code
    return outflag  File "/var/folders/9_/tjgwhcvn6nq39lnwk8yyqhwc0000gn/T/ipykernel_14810/4016639901.py", line 61, in <module>
    X_train = scaler.fit_transform(batch_forward( total_graph['node_feats'], training_nodes, K, model, neighborhoods).numpy())  File "/var/folders/9_/tjgwhcvn6nq39lnwk8yyqhwc0000gn/T/ipykernel_14810/721138015.py", line 70, in batch_forward
    conc = tf.concat([hN, hv], axis=0)  File "/opt/anaconda3/envs/venv/lib/python3.7/site-packages/tensorflow/python/util/tf_should_use.py", line 245, in wrapped
    error_in_function=error_in_function)


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/opt/anaconda3/envs/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3572, in run_code
    return outflag  File "/var/folders/9_/tjgwhcvn6nq39lnwk8yyqhwc0000gn/T/ipykernel_14810/4016639901.py", line 61, in <module>
    X_train = scaler.fit_transform(batch_forward( total_graph['node_feats'], training_nodes, K, model, neighborhoods).numpy())  File "/var/folders/9_/tjgwhcvn6nq39lnwk8yyqhwc0000gn/T/ipykernel_14810/721138015.py", line 70, in batch_forward
    conc = tf.concat([hN, hv], axis=0)  File "/opt/anaconda3/envs/venv/lib/python3.7/site-packages/tensorflow/python/util/tf_should_use.py", line 245, in wrapped
    error_in_function=error_in_function)


In [4]:

#only consider node degree for now
# node_features = np.zeros((total_graph['N_nodes'],2))
# for i in range(total_graph['N_nodes']):
#     degree = len(total_graph['edges'][i+1])
#     node_features[i,0] = degree
#     node_features[i,1] = total_graph['groups'][i+1][0]

def get_similarity(total_graph):
    """Construct a dict of similar nodes, i.e. ones that are within distance 2 of each other"""
    similarity_dict = {i:set() for i in range(total_graph['N_nodes'])}
    for i in range(total_graph['N_nodes']):
        if i%1000==0:
            print(i)
        nb = total_graph['edges'][i]
        similarity_dict[i].update(nb)
        for n in nb:
            new_neighbors = total_graph['edges'][n]
            similarity_dict[i].update(new_neighbors)
            #for j in new_neighbors:
            #    similarity_dict[i].update(total_graph['edges'][j])       
    return similarity_dict

sim_dict = get_similarity(total_graph)

def get_positive_samples_old(edge_dict, nodes, sim_dict):
    """perform random walk starting from each node, and sample 1 from the nodes that occurred """
    walk_length = 5
    walk = np.zeros((5), dtype=int)
    N_nodes = len(nodes)
    co_occuring_ind = np.zeros((N_nodes), dtype=int)
    for ind, v in enumerate(nodes):
        neighbors = edge_dict[v]
        for i in range(walk_length):
            new_node = random.choice(neighbors)
            neighbors = edge_dict[new_node]
            walk[i] = new_node
        u = random.choice(walk)
        co_occuring_ind[ind] = u
    return co_occuring_ind

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000


In [11]:

def get_degree_count(edge_dict, N_nodes):
    degree_count = {i:0 for i in range(N_nodes)}
    for i in range(N_nodes):
        nb_count = len(edge_dict[i]) 
        degree_count[i] = nb_count
    return degree_count

def concat(a, b):
    return tf.concat([a, b], axis=0)


def get_positive_samples(edge_dict, nodes, sim_dict):
    """perform random walk starting from each node, and sample 1 from the nodes that occurred """
    N_nodes = len(nodes)
    co_occuring_ind = []
    for i, v in enumerate(nodes):
        for j,u in enumerate(nodes):
            if u in sim_dict[v]:
                co_occuring_ind.append(j)
                break       
    return co_occuring_ind

def get_negative_samples(total_graph, nodes, Q):
    batch_size = len(nodes)
    batch_indices = [i for i in range(batch_size)]
    neg_ind = np.zeros((batch_size, Q), dtype=int)     
    for j in range(Q):
        for i, v in enumerate(nodes):
            found_negative_edge = False
            while not found_negative_edge:
                ind = random.choice(batch_indices)
                node = nodes[ind]
                if not total_graph['adj_matrix'][node, v] and not total_graph['adj_matrix'][v, node]:
                    neg_ind[i,j] = ind
                    found_negative_edge = True
    return neg_ind

def compute_neighborhoods(edge_dict, N_nodes, nb_size, degree_count):
    neighborhoods = [[] for i in range(N_nodes)]
    for v, neighbors in edge_dict.items():
        nb = len(neighbors)
        sample_size = tf.minimum(nb_size, nb)
        if sample_size==1:
            sample_neighborhood = [edge_dict[v][0]]
        else:
            neighborhood_ind = tf.random.uniform((sample_size,), maxval=nb-1, dtype=tf.int32)
            sample_neighborhood = tf.gather(neighbors, neighborhood_ind).numpy().tolist()
        neighborhoods[v] = sample_neighborhood
    return neighborhoods


def batch_forward(node_features, batch_nodes, K, model, neighborhoods):
    eps = 1e-9
    B = [[] for k in range(K)]
    B[K-1] = batch_nodes[:]
    for k in range(K-1, 0, -1):
        B[k-1] = B[k][:]
        for node in batch_nodes:
            B[k-1].extend(neighborhoods[node])
            
    h = node_features
    N_nodes = h.shape[0]
    h_updated = tf.TensorArray(dtype=tf.float32, size=N_nodes)
    for k in range(K):
        bk = B[k]
        for i, v in enumerate(bk):        
            neighborhood = neighborhoods[v]
            hv = tf.reshape(h[v], (1, -1))
            hN = tf.gather(h, neighborhood)
            conc = tf.concat([hN, hv], axis=0)
            aggregated = tf.cast(tf.reduce_mean(conc, axis=0, keepdims=True), dtype=tf.float32)
            layer = model.layers[k]
            hv = layer(aggregated)
            h_updated = h_updated.write(v, hv)
        
        h = tf.squeeze(h_updated.stack())
        h = h / (tf.norm(h, axis=1, keepdims=True)+eps)
        h_updated = tf.TensorArray(dtype=tf.float32, size=N_nodes)

    return tf.gather(h, batch_nodes)


def compute_embedding(node_features, N_nodes, edge_dict, K, model):
    degree_count = get_degree_count(edge_dict, N_nodes)
    fixed_neighborhood_size = 5
    h = node_features
    h_updated = tf.TensorArray(dtype=tf.float32, size=N_nodes)
    for k in range(K):
        for v in range(N_nodes):
            if fixed_neighborhood_size>degree_count[v]:
                neighborhood = [edge_dict[v][0]]*fixed_neighborhood_size
            else:
                neighborhood_ind = tf.random.uniform((fixed_neighborhood_size,), maxval=degree_count[v]-1, dtype=tf.int32)
                neighborhood = tf.gather(edge_dict[v], neighborhood_ind)
            hv = tf.reshape(h[v], (1, -1))
            hN = tf.gather(h, neighborhood)
            hv = model(tf.reduce_mean(tf.concat([hN, hv], axis=0), axis=0, keepdims=True))
            h_updated = h_updated.write(v, tf.reshape(hv, [-1]))
        h = h_updated.stack()
        h = h / tf.norm(h, axis=1, keepdims=True)
        h_updated = tf.TensorArray(dtype=tf.float32, size=N_nodes)
    return h


def compute_loss(Z, total_graph, nodes, Q=5, sim_dict=None):
    positive_samples_ind = get_positive_samples(total_graph['edges'], nodes, sim_dict)  # has shape (N_nodes,)
    negative_samples_ind = get_negative_samples(total_graph, nodes, Q)    # has shape (N_nodes, Q)  
    Z_pos = tf.gather(Z, positive_samples_ind)
    dot = tf.reduce_sum(tf.multiply(Z, Z_pos), axis=1)
    term1 = -tf.math.log(tf.math.sigmoid(dot))
    term2 = 0
    for i in range(Q):
        Z_neg = tf.gather(Z, negative_samples_ind[:,i])
        term2 -= tf.math.log(tf.math.sigmoid(-tf.reduce_sum(tf.multiply(Z, Z_neg), axis=1)))
    loss = term1 + term2
    return tf.reduce_sum(loss)


In [28]:


def split_into_random_batches(my_list, N):
    random.shuffle(my_list)  # Shuffle the list in-place
    return [my_list[i:i + N] for i in range(0, len(my_list), N)]

feature_dim = total_graph['node_feats'].shape[1]
output_dim = 32
K = 2
Q = 10
fixed_nb_size = 4
num_epochs = 1
batch_size = 256
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001, epsilon=1e-7)

# define model
model = Sequential()
model.add(tf.keras.Input(shape=(feature_dim,)))
for k in range(K):
    model.add(Dense(units=output_dim, activation='relu', bias_initializer="zeros"))
model(total_graph['node_feats'][0].reshape((1,50)))


loss_over_epochs = []
for epoch in range(num_epochs):
    neighborhoods = compute_neighborhoods(total_graph['edges'], total_graph['N_nodes'], fixed_nb_size, get_degree_count(total_graph['edges'], total_graph['N_nodes']))
    batches = split_into_random_batches(total_graph['nodes'], batch_size)
    print(len(batches))
    for i, batch_nodes in enumerate(batches):
        with tf.GradientTape() as tape:
            # Forward pass
            
            Z = batch_forward(total_graph['node_feats'], batch_nodes, K, model, neighborhoods)
            
            # Calculate the loss
            loss = compute_loss(Z, total_graph, batch_nodes, Q, sim_dict)
            print(i,loss)

        # Calculate gradients
        gradients = tape.gradient(loss, model.trainable_variables)
     
        # Update model weights
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    loss_over_epochs.append(loss)    

58
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/opt/anaconda3/envs/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3572, in run_code
    return outflag  File "/var/folders/9_/tjgwhcvn6nq39lnwk8yyqhwc0000gn/T/ipykernel_14810/2464680125.py", line 61, in <module>
    X_train = scaler.fit_transform(batch_forward( total_graph['node_feats'], training_nodes, K, model, neighborhoods).numpy())  File "/var/folders/9_/tjgwhcvn6nq39lnwk8yyqhwc0000gn/T/ipykernel_14810/721138015.py", line 69, in batch_forward
    hN = tf.gather(h, neighborhood)  File "/opt/anaconda3/envs/venv/lib/python3.7/site-packages/tensorflow/python/util/tf_should_use.py", line 245, in wrapped
    error_in_function=error_in_function)


ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/opt/anaconda3/envs/venv/lib/python3.7/site-packages/IPython/core/interactiveshell.py", line 3572, in run_code
    return outflag  File "/var/folders/9_/tjgwhcvn6nq39lnwk8yyqhwc0000gn/T/ipykernel_14810/2464680125.py", line 61, in <module>
    X_train = scaler.fit_transform(batch_forward( total_graph['node_feats'], training_nodes, K, model, neighborhoods).numpy())  File "/var/folders/9_/tjgwhcvn6nq39lnwk8yyqhwc0000gn/T/ipykernel_14810/721138015.py", line 69, in batch_forward
    hN = tf.gather(h, neighborhood)  File "/opt/anaconda3/envs/venv/lib/python3.7/site-packages/tensorflow/python/util/tf_should_use.py", line 245, in wrapped
    error_in_function=error_in_function)


0 tf.Tensor(2900.5698, shape=(), dtype=float32)
1 tf.Tensor(2882.1663, shape=(), dtype=float32)
2 tf.Tensor(2782.7014, shape=(), dtype=float32)
3 tf.Tensor(2720.6172, shape=(), dtype=float32)
4 tf.Tensor(2736.8096, shape=(), dtype=float32)
5 tf.Tensor(2675.27, shape=(), dtype=float32)
6 tf.Tensor(2642.4268, shape=(), dtype=float32)
7 tf.Tensor(2597.4336, shape=(), dtype=float32)
8 tf.Tensor(2576.5996, shape=(), dtype=float32)
9 tf.Tensor(2536.595, shape=(), dtype=float32)
10 tf.Tensor(2519.543, shape=(), dtype=float32)
11 tf.Tensor(2500.136, shape=(), dtype=float32)
12 tf.Tensor(2479.4822, shape=(), dtype=float32)
13 tf.Tensor(2443.276, shape=(), dtype=float32)
14 tf.Tensor(2417.5837, shape=(), dtype=float32)
15 tf.Tensor(2411.29, shape=(), dtype=float32)
16 tf.Tensor(2392.9, shape=(), dtype=float32)
17 tf.Tensor(2370.1404, shape=(), dtype=float32)
18 tf.Tensor(2346.3213, shape=(), dtype=float32)
19 tf.Tensor(2319.8677, shape=(), dtype=float32)
20 tf.Tensor(2306.5684, shape=(), dtype=f

In [36]:

NC_5folds = {}
kf = KFold(n_splits=5, shuffle=True)
nodes = np.array([i for i in range(total_graph['N_nodes'])])
for i, (train_index, test_index) in enumerate(kf.split(nodes)):  
    NC_5folds[i] = {"train":list(nodes[train_index]), "test":list(nodes[test_index])}


In [37]:
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from  sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import StandardScaler


mb = MultiLabelBinarizer(classes=[i for i in range(121)])
scaler = StandardScaler()

def onehot(y, nclasses):
    Y = np.zeros((y.shape[0], nclasses), dtype=int)
    for i in range(y.shape[0]):
        c = y[i]
        Y[i,c-1] =  1
    return Y


def precision_and_recall(Y_true, Y_pred, nclasses):
    # count true positives and false positives and false negatives
    #nclasses = len(Y_true[0])
    TP_list = [0]*nclasses
    FP_list = [0]*nclasses
    FN_list = [0]*nclasses
    for j in range(nclasses):
       for i, pred in enumerate(Y_pred):
            if pred[j]==1 and Y_true[i][j]==1:
                TP_list[j] += 1
            elif pred[j]==1 and  Y_true[i][j]==0:
                FP_list[j] += 1
            elif pred[j]==0 and Y_true[i][j]==1:
                FN_list[j] += 1 

    return TP_list, FP_list, FN_list

def compute_f1_macro(Y_true, Y_pred, nclasses):
    #nclasses = len(Y_true[0])
    TP_list, FP_list, FN_list = precision_and_recall(Y_true, Y_pred, nclasses)
    f1_scores = [0]*nclasses
    for k in range(nclasses):
        if TP_list[k]==0:
            continue
        f1_scores[k] = TP_list[k]/(TP_list[k]+0.5*(FP_list[k]+FN_list[k])) 
    return np.sum(f1_scores)/nclasses


def compute_f1_micro(Y_true, Y_pred, nclasses):
    TP_list, FP_list, FN_list = precision_and_recall(Y_true, Y_pred, nclasses)
    TP = np.sum(TP_list)
    FP = np.sum(FP_list)
    FN = np.sum(FN_list)
    return TP/(TP + 0.5*(FN+FP))

f1_macro_list = []
f1_micro_list = []

# 5-fold cross validation
for i in range(1):
    print(i)
    training_nodes = NC_5folds[i]['train']
    test_nodes = NC_5folds[i]['test']
    X_train = scaler.fit_transform(batch_forward(total_graph['node_feats'], training_nodes, K, model, neighborhoods).numpy())
    X_test = scaler.fit_transform(batch_forward(total_graph['node_feats'], test_nodes, K, model, neighborhoods))
    # For the datasets that only have one one label per node, it gives better results to not use multioutputclassifier
    if not total_graph['Multioutput']:
        Y_train_sequence = np.array([total_graph['groups'][node][0]  for node in training_nodes],dtype=int)
        Y_test_sequence = np.array([total_graph['groups'][node][0] for node in test_nodes], dtype=int)
        log_reg = LogisticRegression(multi_class="ovr", max_iter=200)
        Y_train = Y_train_sequence
        Y_test = Y_test_sequence
        log_reg.fit(X_train, Y_train)
        Y_pred = log_reg.predict(X_test)
        Y_pred = onehot(Y_pred, total_graph['N_classes'])
        Y_test = onehot(Y_test, total_graph['N_classes'])
    else:
        Y_train_sequence = [total_graph['groups'][node]  for node in training_nodes]
        Y_test_sequence = [total_graph['groups'][node] for node in test_nodes]
        Y_train = mb.fit_transform(Y_train_sequence)
        Y_test = mb.fit_transform(Y_test_sequence)
        log_reg = MultiOutputClassifier(LogisticRegression(multi_class="ovr", max_iter=200))
        log_reg.fit(X_train, Y_train)
        Y_pred = log_reg.predict(X_test)
        
    f1_macro = compute_f1_macro(Y_test, Y_pred, total_graph['N_classes'])
    f1_micro = compute_f1_micro(Y_test, Y_pred,total_graph['N_classes'])

    f1_macro_list.append(f1_macro)
    f1_micro_list.append(f1_micro)
    print(f1_macro, f1_micro)
    
print(np.mean(f1_micro_list))
print(np.mean(f1_macro_list))

0
0.7793874172185431 0.7793874172185431
0.7793874172185431
0.7793874172185431
